# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
#llm_config = {"model": "gpt-3.5-turbo"}
import os
config_list = [{"api_type":"groq","model":"llama3-70b-8192","api_key":os.environ.get("GROQ_API_KEY"),}]
llm_config={"config_list" : config_list}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

{'content': '**"Revolutionizing AI Education: The DeepLearning.AI Story"**\n\nDeepLearning.AI is democratizing access to AI education, making it possible for anyone to master the future of technology. Founded by Andrew Ng, a pioneer in AI, this platform offers a range of courses, tutorials, and certifications that cater to diverse skill levels. From beginners to experts, DeepLearning.AI\'s interactive and project-based learning approach helps learners build real-world skills in AI, machine learning, and deep learning. With a mission to make AI accessible to all, DeepLearning.AI is bridging the gap between industry and academia, empowering individuals to transform their careers and shape the future of AI.', 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': None}


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

"Unlocking AI's Full Potential: Introducing DeepLearning.AI"

DeepLearning.AI is revolutionizing the AI landscape by providing a one-stop-shop for all deep learning needs. Founded by Andrew Ng, this innovative platform offers a comprehensive suite of tools, courses, and resources to help individuals and organizations master AI. From beginner-friendly tutorials to advanced research papers, DeepLearning.AI simplifies the AI journey. With its vast community and industry-leading experts, this platform is the ultimate destination for anyone looking to harness the power of AI and stay ahead in the rapidly evolving tech landscape.

--------------------------------------------------------------------------------
Critic (to Writer):

Here'

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

"Unlocking AI's Full Potential: Introducing DeepLearning.AI"

DeepLearning.AI is revolutionizing the AI landscape by providing a one-stop-shop for all deep learning needs. Founded by Andrew Ng, this innovative platform offers a comprehensive suite of tools, courses, and resources to help individuals and organizations master AI. From beginner-friendly tutorials to advanced research papers, DeepLearning.AI simplifies the AI journey. With its vast community and industry-leading experts, this platform is the ultimate destination for anyone looking to harness the power of AI and stay ahead in the rapidly evolving tech landscape.

--------------------------------------------------------------------------------

*************************

## Get the summary

In [16]:
import pprint
pprint.pprint(res.summary)

('Here is the revised blog post:\n'
 '\n'
 '"Unlocking AI\'s Full Potential: Introducing DeepLearning.AI"\n'
 '\n'
 'DeepLearning.AI is revolutionizing the AI landscape by providing a '
 'one-stop-shop for all deep learning needs. Founded by Andrew Ng, a pioneer '
 'in the AI field, this innovative platform offers a comprehensive suite of '
 'tools, courses, and resources to help individuals and organizations master '
 'AI. For those new to AI, deep learning refers to a subfield of machine '
 'learning that involves the use of neural networks to analyze and interpret '
 'data. \n'
 '\n'
 'With its vast community and industry-leading experts, DeepLearning.AI '
 'simplifies the AI journey, providing a platform where users can share '
 'knowledge, learn from each other, and stay updated on the latest '
 'developments in the field. For example, its courses have been taken by over '
 "2.5 million students, demonstrating the platform's ability to reach a wide "
 'audience. By providing a ran